In [9]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import docx2txt
import PyPDF2
import csv
import mailbox
import pandas as pd
from email import policy
from email.parser import BytesParser
import re

# account credentials
#username = "emailtestproject123@gmail.com"
#password = "Testproject@123"

class GMAIL_EXTRACTOR():

    def helloWorld(self):
            print("\nWelcome to Gmail extractor,\ndeveloped by SSRHani.")

    def initializeVariables(self):
            self.usr = ""
            self.pwd = ""
            self.mail = object
            self.mailbox = ""
            self.mailCount = 0
            self.destFolder = ""
            self.data = []
            self.ids = []
            self.idsList = []

    def getLogin(self):
            print("\nPlease enter your Gmail login details below.")
            self.usr = input("Email: ")
            self.pwd = input("Password: ")

    def attemptLogin(self):
        self.mail = imaplib.IMAP4_SSL("imap.gmail.com", 993)
        if self.mail.login(self.usr, self.pwd):
            print("\nLogin SUCCESSFUL")
            #self.destFolder = input("\nPlease choose a destination folder in the form of /Users/username/dest/ (do not forget trailing slash!): ")
            #if not self.destFolder.endswith("/"): self.destFolder+="/"
            return True
        else:
            print("\nLogin FAILED")
            return False

    def checkIfUsersWantsToContinue(self):
        print("\nWe have found "+str(self.mailCount)+" emails in the mailbox "+self.mailbox+".")
        return True if input("Do you wish to continue extracting all the emails into "+self.destFolder+"? (y/N) ").lower().strip()[:1] == "y" else False       

    def selectMailbox(self):
        self.mailbox = input("\nPlease type the name of the mailbox you want to extract, e.g. Inbox: ")
        bin_count = self.mail.select(self.mailbox)[1]
        self.mailCount = int(bin_count[0].decode("utf-8"))
        return True if self.mailCount > 0 else False

    def searchThroughMailbox(self):
        type, self.data = self.mail.search(None, "ALL")
        self.ids = self.data[0]
        self.idsList = self.ids.split()

    def parseEmails(self):

        def clean(text):
            # clean text for creating a folder
            return "".join(c if c.isalnum() else "_" for c in text)


        # create an IMAP4 class with SSL 
        imap = imaplib.IMAP4_SSL("imap.gmail.com")
        # authenticate
        imap.login(self.usr, self.pwd)

        status, messages = imap.select("INBOX")
        # number of top emails to fetch
        N = 5
        # total number of emails
        messages = int(messages[0])
        
        #pass details to details.csv
        with open('details.csv', 'w', newline='') as email_file:
            email_writer = csv.writer(email_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            email_writer.writerow(['no','subject','email','name','isattachmissed','company','in_time','in_date'])
            no = 0
            name = ""
            #for is inside the with----------------------------------------------------------------------------
            for i in range(messages, messages-N, -1):
                
                # fetch the email message by ID
                res, msg = imap.fetch(str(i), "(RFC822)")
                for response in msg:
                    
                    if isinstance(response, tuple):
                        # parse a bytes email into a message object
                        msg = email.message_from_bytes(response[1])
                        # decode the email subject
                        subject, encoding = decode_header(msg["Subject"])[0]
                        if isinstance(subject, bytes):
                            # if it's a bytes, decode to str
                            subject = subject.decode(encoding)
                        # decode email sender
                        From, encoding = decode_header(msg.get("From"))[0]
                        if isinstance(From, bytes):
                            From = From.decode(encoding)
                                 
                            name, encoding = decode_header(re.findall('\S*@\S*\s?', From))[0]
                            if isinstance(name, bytes):
                                name = name.decode(encoding)
                                pytho
                        print("\n")
                        print("\n")
                        print("\n")
                        print("Subject:", subject)
                        print("From:", From)
                        no = no + 1
                        # if the email message is multipart--------------------------------------------------------------------
                        if msg.is_multipart():
                            email_writer.writerow([no,subject,From,(re.findall('^[a-zA-Z]+ [a-zA-Z]+', From)),0,'wso2',10,10])
                            # iterate over email parts
                            for part in msg.walk():
                                #csv file writing for with attachment emails
                                
                                # extract content type of email
                                content_type = part.get_content_type()
                                content_disposition = str(part.get("Content-Disposition"))
                                try:
                                    # get the email body
                                    body = part.get_payload(decode=True).decode()
                                except:
                                    pass
                                if content_type == "text/plain" and "attachment" not in content_disposition:
                                    # print text/plain emails and skip attachments
                                    print("Body1:",body)
                                    
                                    #print("Attachment:","No Attachments in the email")
                                elif "attachment" in content_disposition:
                                    print("Attachment1:","Attachments in the email")
                                    
                                    # download attachment
                                    filename = part.get_filename()
                                    print("Filename:",filename)

                                    if filename:
                                        folder_name = clean(subject)
                                        if not os.path.isdir(folder_name):
                                            # make a folder for this email (named after the subject)
                                            os.mkdir(folder_name)
                                        filepath = os.path.join(folder_name, filename)
                                        # download attachment and save it
                                        open(filepath, "wb").write(part.get_payload(decode=True))
                                        pdfFileObj = open(filepath,'rb')
                                            # Creating a pdf reader object
                                        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
                                        # Getting number of pages in pdf file
                                        pages = pdfReader.numPages
                                        # Loop for reading all the Pages
                                        for i in range(pages):
                                                # Creating a page object
                                                pageObj = pdfReader.getPage(i)
                                                # Printing Page Number
                                                print("Page No: ",i)
                                                # Extracting text from page
                                                # And splitting it into chunks of lines
                                                text = pageObj.extractText().split("  ")
                                                # Finally the lines are stored into list
                                                # For iterating over list a loop is used
                                                for i in range(len(text)):
                                                        # Printing the line
                                                        # Lines are seprated using "\n"
                                                        print(text[i],end="\n\n")
                                                # For Seprating the Pages
                                                print()

                        # if the email message is not  multipart--------------------------------                
                        else:
                            #csv file writing for without attachment emails
                            #email_writer.writerow([no,subject,From,'1'])
                            email_writer.writerow([no,subject,From,(re.findall('^[a-zA-Z]+ [a-zA-Z]+', From)),1,0,0,0])

                            #no = no + 1
                            # extract content type of email
                            content_type = msg.get_content_type()
                            # get the email body
                            body = msg.get_payload(decode=True).decode()
                            if content_type == "text/plain":
                                # print only text email parts
                                print("Body2:",body)
                                print("Attachment:","No Attachments in the email")


                               # creating a data frame
                                #df = pd.DataFrame([[subject, From, '1']], columns = ['subject', 'email', 'isattachmissed' ])

                                # writing data frame to a CSV file
                                #df.to_csv('details.csv')

                       
                        print("="*100)

            
    def __init__(self):
            self.initializeVariables()
            self.helloWorld()
            self.getLogin()
            if self.attemptLogin():
                not self.selectMailbox() and sys.exit()
            else:
                sys.exit()
            not self.checkIfUsersWantsToContinue() and sys.exit()
            self.searchThroughMailbox()
            self.parseEmails()

    

if __name__ == "__main__":
    run = GMAIL_EXTRACTOR()
            
# close the connection and logout
#imap.close()
#imap.logout()




Welcome to Gmail extractor,
developed by SSRHani.

Please enter your Gmail login details below.
Email: researchsenderemail@gmail.com
Password: researchs@123

Login SUCCESSFUL

Please type the name of the mailbox you want to extract, e.g. Inbox: Inbox

We have found 5 emails in the mailbox Inbox.
Do you wish to continue extracting all the emails into ? (y/N) y






Subject: Research Email Test
From: research receiveremail <researchreceiveremail@gmail.com>
Body1: DEAR He,
Find the Attachment

Attachment1: Attachments in the email
Filename: COVER_LETTER_SAHANI_RAJAPAKSHE.pdf
Page No:  0
R P S S Rajapakshe 
 
No: 546
 
Mahanelubewa
 
Hidogama
 
Anuradhapura
 
15
/0
2
/2021
 
 
Hiring Manager
 
 
Dear Sir/Madam
 
 
Applying for 
Associate 
Software 
Engineer
 
vacancies at your company
 
 

Associate Software E
ngineer
 
position with your company. As a 
software engineering undergraduate at the University of Kelaniya with an increasingly extensive 
knowledge of implementations, I feel co

In [ ]:
emailtestproject123@gmail.com
Testproject@123
 /Users/LENOVO/Desktop/Emaildel/

In [ ]:
researchreceiveremail@gmail.com
researchr@123
researchsenderemail@gmail.com
researchs@123

In [ ]:
with open('details.csv', 'w', newline='') as in_email_file,open('details.csv', mode='r') as out_email_file:
                                reader = csv.reader(out_email_file)
                                next(reader, None) #skip header
                                writer = csv.writer(in_email_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                                #email_writer.writerow(['subject','From','isattachmissed'])
                                for row in reader:
                                    writer.writerow([subject,From,'1'])

In [ ]:
 with open('details.csv', 'w', newline='') as email_file:
                                email_writer = csv.writer(email_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                                email_writer.writerow(['subject','From','isattachmissed'])
                                email_writer.writerow([subject,From,'1'])